In [ ]:
from rdflib import Graph, Namespace, URIRef, RDF, RDFS, OWL

In [ ]:
from rdflib.namespace import SDO

In [ ]:
from rdflib import Graph, Namespace, URIRef
import re

g = Graph()
g.parse("WB.owl", format="turtle")

<Graph identifier=Nd84496f3897544d2b22ac90223111f18 (<class 'rdflib.graph.Graph'>)>

In [ ]:
def retrieve(expression):
    matches = re.findall(r'(\b\w+)\s*:\s*(Q\d+|P\d+|Statement|Reference)(?:\([^\)]*\))?', expression)
    print(matches)
    seen = set()
    blocks = []

    # namespace
    namespace_map = {
        "ed": Namespace("http://example.org/entity/"),
        "ep": Namespace("http://example.org/prop/"),
        "eps": Namespace("http://example.org/prop/statement/"),
        "edt": Namespace("http://example.org/prop/direct/"),
        "edv": Namespace("http://example.org/value/"),
        "epq": Namespace("http://example.org/prop/qualifier/"),
        "epr": Namespace("http://example.org/prop/reference/"),
        "wikibase": Namespace("http://wikiba.se/ontology#"),
        "schema": Namespace("http://schema.org/"),

    }

    # statement and reference
    special_classes = {
        "Statement": "wikibase:Statement rdf:type owl:Class .",
        "Reference": "wikibase:Reference rdf:type owl:Class ."
    }

    for match in matches: # Iterate through the matches
        prefix = match[0] # Extract the prefix from the match
        identifier = match[1] # Extract the identifier from the match

        key = f"{prefix}:{identifier}"
        if key in seen:
            continue
        seen.add(key)

        if prefix not in namespace_map:
            continue

        if prefix == "wikibase" and identifier in special_classes:
            blocks.append(
                f"###  http://wikiba.se/ontology#{identifier}\n"
                f"{special_classes[identifier]}"
            )
            continue

        subject = namespace_map[prefix][identifier]
        triples = list(g.predicate_objects(subject))
        if not triples:
            blocks.append(f"###  {subject}\n{prefix}:{identifier} .")
            continue

        lines = [f"###  {subject}", f"{prefix}:{identifier}"]
        for i, (p, o) in enumerate(triples):
            pred_str = (
                "rdf:type" if p == RDF.type else
                "rdfs:label" if p == RDFS.label else
                 "schema:description" if p == SCHEMA.description else
                f"<{p}>"
            )
            if isinstance(o, URIRef):
                obj_str = (
                    "owl:" + str(o).split("/")[-1] if str(o).startswith(str(OWL)) else
                    f"<{o}>"
                )
            else:
                obj_str = f"\"{o}\""
            end_char = " ;" if i < len(triples) - 1 else " ."
            lines.append(f" {pred_str} {obj_str}{end_char}")
        blocks.append('\n'.join(lines))

    return blocks

In [ ]:
# Namespaces
ED = Namespace("https://lod.enslaved.org/entity/")
EP = Namespace("https://lod.enslaved.org/prop/")
EDS = Namespace("https://lod.enslaved.org/entity/statement/")
EDT = Namespace("https://lod.enslaved.org/prop/direct/")
EDV = Namespace("https://lod.enslaved.org/value/")
EPQ = Namespace("https://lod.enslaved.org/prop/qualifier/")
EPR = Namespace("https://lod.enslaved.org/prop/reference/")
EPS = Namespace("https://lod.enslaved.org/prop/statement/")
SCHEMA = Namespace("http://schema.org/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
OWL = Namespace("http://www.w3.org/2002/07/owl#")
RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
WIKIBASE = Namespace("http://wikiba.se/ontology#")

In [ ]:
import os
import openai
import subprocess
import pandas as pd
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI(api_key='s...')

# === Step 1: Read input files ===

with open("drive/MyDrive/EnslavedOM/enslaved_V2.txt", "r") as f:
    module_file_read = f.read()


with open("wikibase_triple.ttl", "r") as f:
     WiBa_ttl_read = f.read()
def get_alignment_rule(WiBa_ttl_read, f):
# === Step 2: Construct the prompt ===
      prompt = f"""
      We have two ontologies: Enslaved and Wikibase.
      We need to find the complex ontology alignment rules between these two ontologies.

      All the information regarding the modules and patterns of the Enslaved ontology is provided here:
      {module_file_read}

      Now, consider the following triple from the Wikibase ontology:
      Wikibase triple:
      {WiBa_ttl_read}

      Now, find the alignment rules for the given Wikibase triple with respect to the Enslaved ontology, based on the module information provided. Provide the output in the format of Alignment Rules: the generated alignment rulles.

      """

      # === Step 3: Query OpenAI API ===
      response = client.chat.completions.create(
          model="gpt-4o",
          messages=[{"role": "user", "content": prompt}],
          temperature=0,
          top_p=1
      )

      output_text = response.choices[0].message.content.strip()

      # === Step 4: Display and Save Output ===
      print("Alignment Rule Output:\n")
      print(output_text)
      output_dir = '/content/drive/MyDrive/EnslavedOM/Outputs1'

      if not os.path.exists(output_dir):
          os.makedirs(output_dir)

      # Save to file
      with open(f"{output_dir}/alignment_output_{f}.txt", "w", encoding="utf-8") as out_file:
          out_file.write(output_text)

      print(f"\nOutput saved!'{f}")


df = pd.read_csv('EnslavedR.csv')
col_names = df.columns
print(col_names)
for name in col_names:
  print("col name",name)
  cname = df[name]
  for i in range (cname.count()):
      print(cname[i])
      f = str(name)+"_"+str(i+1)
      b = retrieve(cname[i])
      print("expressions",b)
      get_alignment_rule(b, f)

Streaming output truncated to the last 5000 lines.

### Wikibase Triple Breakdown:
1. **Person Class:**
   - `http://example.org/entity/Q410` is labeled as "Person".
   - In the Enslaved ontology, `Person` is a subclass of `Agent`.

2. **hasAgeRecord Property:**
   - `http://example.org/prop/P42` is labeled as "hasAgeRecord".
   - The domain is `http://example.org/entity/Q410` (Person).
   - The range is `http://wikiba.se/ontology#Statement`.
   - In the Enslaved ontology, `hasAgeRecord` is a subproperty of `hasPersonRecord`.

3. **Statement Class:**
   - `http://wikiba.se/ontology#Statement` is a class.
   - In the Enslaved ontology, `AgentRecord` is a class that can be associated with `Person`.

4. **age value Property:**
   - `http://example.org/prop/statement/P42` is labeled as "age value".
   - The domain is `http://wikiba.se/ontology#Statement`.
   - The range is `http://example.org/entity/Q424` (AgeRecord).
   - In the Enslaved ontology, `AgeRecord` is a subclass of `AgentRecord